# Controlled Scenario

This notebook contains the experiment to **reproduce the observed course of Sars-CoV-2 under the taken interventions**, which is reported on Figure 3 in the [paper](https://arxiv.org/abs/2004.07641).

The notebook is organized as follows:

* In Section 1, we define all the simulation parameters.
* In Section 2, we run all the simulations needed for the experiment.
* In Section 3, we plot the results.

To just regenerate the figures from a summary file containing all the simulation objects of the experiment, you can skip Section 2, and only run the cells in Sections 1 & 3. 

---

## 1. Define the experiment parameters

#### Import libs

In [1]:
import numpy as np
import pickle
import multiprocessing

In [2]:
from lib.mobilitysim import MobilitySimulator
from lib.parallel import launch_parallel_simulations
from lib.distributions import CovidDistributions
from lib.data import collect_data_from_df
from lib.measures import (MeasureList, Interval,
                          BetaMultiplierMeasureByType,
                          SocialDistancingForAllMeasure, 
                          SocialDistancingForPositiveMeasure,
                          SocialDistancingForPositiveMeasureHousehold)
from lib.inference import gen_initial_seeds, extract_seeds_from_summary

#### Set the random seed for reproducibility

In [3]:
# Choose random seed
c = 0
# Set it
np.random.seed(c)
# Define prefix string used to save plots
runstr = f'run{c}_'

#### Set the number of roll-outs to simulate

In [4]:
random_repeats = 4 # Set to at least 20 to obtain stable results

#### Define the number of parallel workers used for simulations 

In [5]:
num_workers = multiprocessing.cpu_count() - 1

#### Set the time to simulate

In [6]:
days_until_lockdown = 13 # March 10 - March 23
days_present = 35  # Days since March 10 on April 12 (March 10 is time 0 for our experiments)
max_time_present = 24.0 * (days_present) # in hours

#### Define population/mobility parameters

Import downsampled version of Tübingen (population downsampled by 20x, sites by 10x) that was used used to generate mobility traces on the fly for inference (sites, home locations, etc)

In [7]:
case_downsample = 10                                
mob_settings = 'lib/mobility/Tubingen_settings_10_10_hh.pk'  
area = 'TU'
country = 'GER'



# See town-generator.ipynb for an example on how to create the settings
with open(mob_settings, 'rb') as fp:
    obj = pickle.load(fp)
mob = MobilitySimulator(**obj)

#### Load the case data

Adjust the number of cases to the down-scaling of the population

In [8]:
new_cases_ = collect_data_from_df(country=country, area=area, datatype='new',
    start_date_string='2020-03-10', until=days_present)
new_cases = np.ceil(1/case_downsample * new_cases_)

# Instantiate correct state transition distributions (estimated from in literature)
distributions = CovidDistributions(country=country)

#### Define initial seed count per state 

These numbers are based on the *downscaled* infection counts on March 10.

In [9]:
initial_seeds = gen_initial_seeds(new_cases)

#### Set epidemic parameters

Inferred using Bayesian optimization.

In [11]:
beta = 0.91 # this needs to be calibrated
inferred_params = {
    'betas' : {'education': beta,'social': beta,'bus_stop': beta,'office': beta,'supermarket': beta}, 
    'beta_household' : 0.1 # this needs to be calibrated
}

#### Define standard testing parameters

The parameters are the same used for inference.

In [12]:
def standard_testing(max_time):
    standard_testing_params = {
        'testing_t_window'    : [0.0, max_time], # in hours
        'testing_frequency'   : 24.0,     # in hours
        'test_reporting_lag'  : 48.0,     # in hours (actual and self-report delay)
        'tests_per_batch'     : 10,       # assume 300 tests/day in LK Tübingen
        'test_fpr'            : 0.0,      # test false positive rate
        'test_fnr'            : 0.0,      # test false negative rate
        'test_smart_delta'    : 24.0 * 3, # in hours
        'test_smart_duration' : 24.0 * 7, # in hours
        'test_smart_action'   : 'isolate', 
        'test_smart_num_contacts'   : 10, 
        'test_targets'        : 'isym',
        'test_queue_policy'   : 'fifo',
        'smart_tracing'       : None, 
    }
    return standard_testing_params

#### Define helper I/O functions to load/save the output of the experiment

In [13]:
def save_summary(summary, filename):
    with open('summaries/' + filename, 'wb') as fp:
        pickle.dump(summary, fp)
    
def load_summary(filename):
    with open('summaries/' + filename, 'rb') as fp:
        summary = pickle.load(fp)
    return summary

---

## 2. Run the simulations

Define a helper function to run general type of experiment, fixing all parameters above.

In [14]:
def run(tparam, measure_list, t, local_seeds, dynamic_tracing=False):

    # add standard measure of positives staying isolated
    measure_list +=  [
        SocialDistancingForPositiveMeasure(
            t_window=Interval(0.0, t), p_stay_home=1.0),
        
        SocialDistancingForPositiveMeasureHousehold(
            t_window=Interval(0.0, t), p_isolate=1.0)
    ]
    measure_list = MeasureList(measure_list)

    # run simulations
    summary = launch_parallel_simulations(
        mob_settings, 
        distributions, 
        random_repeats, num_workers, 
        inferred_params, local_seeds, tparam, measure_list, 
        max_time=t, 
        num_people=mob.num_people, 
        num_sites=mob.num_sites, 
        site_loc=mob.site_loc, 
        home_loc=mob.home_loc,
        dynamic_tracing=dynamic_tracing,
        verbose=False)
    return summary
    

Run the experiment using the parameters defined above and simulate in the future, with additional measures of varying duration.

**WARNING: this cell might take a long time to run depending of the parameters defined above!**

In [15]:
tuned_p_stay_home = 0.55 # this needs to be calibrated
tuned_site_multipliers = {'education': 0.0, 'social': 0.0, 'bus_stop': 1.0, 'office': 0.0, 'supermarket': 1.0}

observed_measures_SD = [
    # education close, social closes
    BetaMultiplierMeasureByType(
        t_window=Interval(24.0 * days_until_lockdown, max_time_present), 
        beta_multiplier=tuned_site_multipliers),
    
    # less activities of all due to contact constraints
    SocialDistancingForAllMeasure(
     t_window=Interval(24.0 * days_until_lockdown, max_time_present), 
        p_stay_home=tuned_p_stay_home)
]

testing_params_SD_0 = standard_testing(max_time_present)

summary_SD_0 = run(testing_params_SD_0, observed_measures_SD, max_time_present, initial_seeds)

save_summary(summary_SD_0, 'summary_SD_0.pk')

KeyboardInterrupt: 

## 3. Plot the results

Import plotting utility

In [ ]:
from lib.plot import Plotter
%matplotlib inline

Load the summary holding the results of the experiment to plot.

In [ ]:
summary_SD_0 = load_summary('summary_SD_0.pk')

Reproduce Figure 2 (a) in the paper.

In [ ]:
plotter = Plotter()
plotter.plot_positives_vs_target(
    summary_SD_0, new_cases.sum(axis=1), 
    test_lag=2,
    title='Predictions for observed controlled scenario after March 23', 
    filename=runstr + 'SD_000',
    figsize=(6, 4),
    start_date='2020-03-10',
    errorevery=1, acc=1000, 
    lockdown_at=days_until_lockdown,
    ymax=250)



Reproduce Figure 2 (b) in the paper.

In [ ]:
plotter = Plotter()
plotter.plot_daily_rts(
    summary_SD_0,
    filename=runstr + 'SD_003',
    start_date='2020-03-10',
    sigma=None,
    figsize=(6, 4), 
    subplot_adjust=None,
    lockdown_label='Lockdown', 
    lockdown_at=days_until_lockdown,
    lockdown_label_y=2.6,
    ymax=4.0)

Reproduce Figure 2 (c) in the paper.

In [ ]:
targets = new_cases.sum(axis=1)
targets = np.hstack((targets[0], np.diff(targets)))

plotter = Plotter()
plotter.plot_daily_infected(
    summary_SD_0,
    title='', 
    filename=runstr + 'SD_001',
    errorevery=100, acc=1000, 
    figsize=(6, 4),
    start_date='2020-03-10',
    show_target=targets,
    lockdown_at=days_until_lockdown,
    lockdown_label_y=140,
    lockdown_label='Lockdown',
    ymax=200)

Reproduce Figure 2 (d) in the paper.

In [ ]:
plotter = Plotter()
plotter.plot_cumulative_infected(
    summary_SD_0,
    title='', 
    filename=runstr + 'SD_002',
    figsize=(6, 4),
    legend_loc='upper left',
    errorevery=100, acc=1000, 
    start_date='2020-03-10',
    lockdown_at=days_until_lockdown,
    lockdown_label_y=140,
    show_target=new_cases.sum(axis=1),
    ymax=None)